<a href="https://colab.research.google.com/github/sekharsekhar/ChatBot/blob/main/copy_of_fake_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas scikit-learn joblib


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import re
import joblib
import string


In [ ]:
fake = pd.read_csv("/content/sample_data/Fake (1).csv", on_bad_lines="skip", engine='python')

print(fake.head())
print(fake.info())

                                               title  \
0   Donald Trump Sends Out Embarrassing New Year’...   
1   Drunk Bragging Trump Staffer Started Russian ...   
2   Sheriff David Clarke Becomes An Internet Joke...   
3   Trump Is So Obsessed He Even Has Obama’s Name...   
4   Pope Francis Just Called Out Donald Trump Dur...   

                                                text subject  \
0  Donald Trump just couldn t wish all Americans ...    News   
1  House Intelligence Committee Chairman Devin Nu...    News   
2  On Friday, it was revealed that former Milwauk...    News   
3  On Christmas day, Donald Trump announced that ...    News   
4  Pope Francis used his annual Christmas Day mes...    News   

                date  
0  December 31, 2017  
1  December 31, 2017  
2  December 30, 2017  
3  December 29, 2017  
4  December 25, 2017  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 403 entries, 0 to 402
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 

In [ ]:
true = pd.read_csv("/content/sample_data/True.csv", on_bad_lines="skip", engine='python')

print(true.head())

                                               title  \
0  As U.S. budget fight looms, Republicans flip t...   
1  U.S. military to accept transgender recruits o...   
2  Senior U.S. Republican senator: 'Let Mr. Muell...   
3  FBI Russia probe helped by Australian diplomat...   
4  Trump wants Postal Service to charge 'much mor...   

                                                text       subject  \
0  WASHINGTON (Reuters) - The head of a conservat...  politicsNews   
1  WASHINGTON (Reuters) - Transgender people will...  politicsNews   
2  WASHINGTON (Reuters) - The special counsel inv...  politicsNews   
3  WASHINGTON (Reuters) - Trump campaign adviser ...  politicsNews   
4  SEATTLE/WASHINGTON (Reuters) - President Donal...  politicsNews   

                 date  
0  December 31, 2017   
1  December 29, 2017   
2  December 31, 2017   
3  December 30, 2017   
4  December 29, 2017   


In [ ]:
fake['class']=0
true['class']=1

In [ ]:
data=pd.concat([fake,true],axis=0)
data = data.sample(frac=1, random_state=42).reset_index(drop=True) # Added shuffling and resetting index

In [ ]:
data.sample(10)

,title,text,subject,date,class
860,"Perverted GOP State Rep. Corners Lobbyist, Tr...",The Republican Party is having a terrible week...,News,"November 12, 2017",0
1262,"Exclusive: Contenders emerge for No.2 Fed job,...","(In this Dec. 8 story, corrects Peters’ affil...",politicsNews,"December 8, 2017",1
755,GOP Implodes As California Passes Bill To Mak...,"About a month ago, Donald Trump shocked the co...",News,"September 16, 2017",0
9,Miami Police Union Leader Joins Other Conserv...,"Over 50 people are dead, and more than 500 are...",News,"October 2, 2017",0
731,JPMorgan's Dimon says Trump likely to be a one...,"CHICAGO/NEW YORK (Reuters) - Jamie Dimon, chie...",politicsNews,"November 22, 2017",1
271,House tax positions begin to emerge ahead of t...,WASHINGTON (Reuters) - Republicans in U.S. Hou...,politicsNews,"December 6, 2017",1
580,Russians Fooled Dumbass Conservatives With Me...,If Hillary Clinton got into a fistfight with J...,News,"November 1, 2017",0
653,Two key Republicans agree on national flood in...,WASHINGTON (Reuters) - Two senior U.S. House o...,politicsNews,"November 3, 2017",1
1247,Pompeo might bring assets to 'hellish' secreta...,WASHINGTON (Reuters) - If he becomes U.S. secr...,politicsNews,"November 30, 2017",1
1357,U.S. House speaker: FBI to comply with documen...,WASHINGTON (Reuters) - The FBI had said it wil...,politicsNews,"October 26, 2017",1


In [ ]:
data=data.drop(['title','subject','date'],axis=1)

In [ ]:
data


,text,class
0,Every other day there seems to be a new mass s...,0
1,(Reuters) - The revolving door at the Trump Wh...,1
2,SAN FRANCISCO (Reuters) - A German man has com...,1
3,WASHINGTON (Reuters) - U.S. House Speaker Paul...,1
4,WASHINGTON (Reuters) - The campaign of Roy Moo...,1
...,...,...
1617,BEIJING (Reuters) - Continued stable developme...,1
1618,WASHINGTON (Reuters) - A bipartisan group in t...,1
1619,(Reuters) - A New York comedian has been compe...,1
1620,WASHINGTON (Reuters) - A Senate panel on Tuesd...,1


In [ ]:
data.reset_index(inplace=True)

In [ ]:
data.drop(['index'],axis=1,inplace=True)

In [ ]:
data.sample(5)

,text,class
1362,WASHINGTON/NEW YORK (Reuters) - The White Hous...,1
332,WASHINGTON (Reuters) - Senate Majority Leader ...,1
510,Donald Trump has spent a lot of time cozying u...,0
357,WASHINGTON (Reuters) - Republican U.S. Represe...,1
300,"In the wake of any tragedy in this country, th...",0


In [ ]:
def clean_text(text):
  text=text.lower()
  text=re.sub('\[.*?\]','',text)
  text=re.sub("\\W"," ", text)
  text=re.sub('https?://\S+|www\.\S+','',text)
  text=re.sub('<.*?>+','',text)
  text=re.sub('[%s]' % re.escape(string.punctuation),'',text)
  text=re.sub('\n','',text)
  text=re.sub('\w*\d\w*','',text)
  return text


In [ ]:
data["text"]=data["text"].apply(clean_text)

In [ ]:
x=data["text"]
y=data["class"]

xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.25,random_state=42, stratify=y) # Added stratify

In [ ]:
vectorizer=TfidfVectorizer()
xv_train=vectorizer.fit_transform(xtrain)
xv_text=vectorizer.transform(xtest)

In [ ]:
lr=LogisticRegression()
lr.fit(xv_train,ytrain)

LogisticRegression()

Now that the model is trained, let's evaluate its performance on the test set.

In [ ]:
ypred = lr.predict(xv_text)
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       1.00      0.80      0.89       101
           1       0.94      1.00      0.97       305

    accuracy                           0.95       406
   macro avg       0.97      0.90      0.93       406
weighted avg       0.95      0.95      0.95       406



Finally, let's save the trained model and the TF-IDF vectorizer.

In [ ]:
joblib.dump(lr, 'logistic_regression_model.pkl')
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')

['tfidf_vectorizer.pkl']

Now that the model is trained, let's evaluate its performance on the test set.

In [ ]:
%%writefile app.py
print("Hello, this is my frontend file!")


Writing app.py


In [ ]:
!pip install streamlit pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.4 MB/s eta 0:00:00


In [ ]:
%%writefile demo.py
import streamlit as st
import joblib

vectorizer=joblib.load("vectorizer.jb")
model=joblib.load("model.jb")

st.title("Fake News Detection")
st.write("Enter a text article below to check whether it is fake ir real")
news_input=st.text_area("News Article:")

if st.button("Check News"):
  if news_input:
    vectorized_input=vectorizer.transform([news_input])
    prediction=model.predict(vectorized_input)

    if prediction[0]==1:
      st.success("This news is real!")
    else:
      st.error("This news is fake!")
  else:
    st.warning("Please enter some text to analyze ")

Writing demo.py


In [ ]:
!pip install streamlit pyngrok
!pip install scikit-learn  # If you used sklearn for your model/vectorizer


In [ ]:
!pip install streamlit

In [ ]:
from pyngrok import ngrok
from google.colab import userdata

# Get the authtoken from Colab secrets
ngrok_auth_token = userdata.get('NGROK_AUTH_TOKEN')
if ngrok_auth_token:
    ngrok.set_auth_token(ngrok_auth_token)
else:
    print("Ngrok authtoken not found in Colab secrets. Please add it using the key icon on the left sidebar.")

# Open a public URL for Streamlit
public_url = ngrok.connect(8501)
print("Your Streamlit app will be available at:", public_url)

# Run the app
!streamlit run demo.py --server.port 8501 &

Your Streamlit app will be available at: NgrokTunnel: "https://19ea8dfb4e58.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.86.179.222:8501

  Stopping...


In [ ]:
!git branch


fatal: not a git repository (or any of the parent directories): .git


In [ ]:
!git branch


fatal: not a git repository (or any of the parent directories): .git
